In [ ]:
# Do a pseudo code for vectorize photos of zeros and ones
# use SVD and PCA

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler